In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from math import sqrt
import scipy.stats as stats
from statsmodels.stats.proportion import proportions_ztest

# 1. Funciones muestra y representatividad

In [ ]:
def muestreo(data, porcentaje, semilla = 42):
    numero_muestras = int(len(data) * porcentaje)
    muestra = data.sample(numero_muestras, random_state = semilla)
    return muestra

def representatividad_muestra_target_categorica(data, muestra, target_categorica):

    fig,axs = plt.subplots(1,2,figsize = (10,8))

    freq = data[target_categorica].value_counts(normalize = True)
    axs[0].bar(x = freq.index, height = freq.values, label = "Poblacion")
    axs[0].set_title("superviviencia poblacion")
    freq_m = muestra[target_categorica].value_counts(normalize = True)
    axs[1].bar(x = freq_m.index, height = freq_m.values, label = "Muestra")
    axs[1].set_title("supervivencia muestra")


def representatividad_muestra_target_numerica(data, muestra):

    data_medidas = data.describe()
    muestra_medidas = muestra.describe()
    return data_medidas.merge(muestra_medidas, left_index= True, right_index= True, suffixes = ["_pob","_muestra"]).loc["mean"].T



# 2. Medidas estadísticas variables numéricas

In [ ]:
#============================
# Media muestral
#============================
def IC_media_muestral(muestra, target_numerica):
    s = muestra[target_numerica].std()
    SE = s/sqrt(len(muestra))
    media_muestral = round(muestra[target_numerica].mean(),2)
    IC = round(1.96 * SE,2)
    print("Media muestral:", media_muestral)
    print("IC = ", IC)
    print(f"Intervalo de confianza: [{media_muestral -IC},{media_muestral +IC}]")


#============================
# T-test para media muestral
#============================
def t_test_media_muestral(muestra, target_numerica, media_poblacional, p_valor_deseado = 0.05):
    '''
    Test para saber si podemos aceptar una media poblacional dada
    '''
    t_stat, p_valor = stats.ttest_1samp(muestra[target_numerica], popmean=media_poblacional)

    if p_valor < p_valor_deseado:
        print(f"Se rechaza H0: la media es distinta de {media_poblacional}")
    else:
        print(f"No se rechaza H0: no hay evidencia de que la media sea distinta de {media_poblacional}")

#============================
# T-test intervalo de confianza para media muestral
#============================
def t_test_IC_media_muestral(muestra, target_numerica, p_valor_deseado = 0.05, IC_inicial = 0, IC_final = 5):
    '''
    Test para encontrar el IC de la media muestral dado que queramos alcanzar un p-valor
    '''
    for IC in np.arange(IC_inicial,IC_final,0.05): # partimos de IC de 0 años (a = media muestral) y vamos subiendo hasta 1.5, 
                                        # hasta encontrar el IC máximo, que nos garantice al 95% que la media puede ser ese valor
        a = muestra[target_numerica].mean() + IC # La curva es simétrica nos vale con testar un lado solo
        p_value  = stats.ttest_1samp(a=muestra[target_numerica].values,popmean= a).pvalue
        print(f"Para un IC: {IC}, comprobamos a = {a} y nos da un p-value = {p_value}")
        if p_value <= p_valor_deseado:
            print(f"para IC =  {IC} se rechaza la hipótesis nula de igualdad, es decir a = {a}",\
                f"ya no es una posible media poblacional al {(1-p_valor_deseado)*100}% de signifcacion")
            print(f"Nuestro IC escogido es entonces, IC = {IC_inicial}")
            break
        IC_inicial = IC

#============================
# T-test para umbrales > o <
#============================
# Lo que quiera confirmar lo pongo como hipótesis alternativa Ha.
# Si el p-valor < 0.05, rechazamos la hipótesis nula Ho, y aceptamos la alternativa Ha.

def t_test_umbrales(muestra, target_numerica, media, mayor_o_menor, umbral = 0.05):
    '''
    Lo que quiera confirmar lo pongo como hipótesis alternativa Ha.
    Si el p-valor < 0.05, rechazamos la hipótesis nula Ho, y aceptamos la alternativa Ha.
    target_numerica es el nombre de la columna de la muestra que quiero estudiar
    media es el número que quiero comprobar si la media es mayor o menor que dicho número
    mayor_o_menor indica si la hipótesis alternativa es con > o <, y hay que poner las palabras "greater" o "less"    
    '''
    p_valor = stats.ttest_1samp(a=muestra[target_numerica],popmean=media,alternative=mayor_o_menor).pvalue
    if p_valor < umbral:
        print(f"P-valor {p_valor}, por tanto, se acepta la hipótesis alternativa, la media poblacional es {mayor_o_menor} que {media}")
    else:
        print(f"P-valor {p_valor}, por tanto, no se puede aceptar la hipótesis alternativa, no podemos decir que la media poblacional es {mayor_o_menor} que {media}")

#============================
# T-test para dos muestras
#============================
# Cuando quiero comprobar si hay una media significativa aplicable a dos muestras distintas
# Ho no hay diferencia entre las medias de las muestras
# Ha sí hay diferencia entre las medias de las muestras

def t_test_dos_muestras(muestra1, muestra2, target_numerica, umbral = 0.05):
    '''
    Cuando quiero comprobar si dos muestras tienen misma media
    Ho no hay diferencia entre las medias de las muestras
    Ha sí hay diferencia entre las medias de las muestras
    target_numerica es el nombre de la columna de la muestra que quiero estudiar
    '''
    p_valor = stats.ttest_ind(a=muestra1[target_numerica],b=muestra2[target_numerica],alternative="two_sided").pvalue
    if p_valor < umbral:
        print(f"P-valor {p_valor}, por tanto, se acepta la hipótesis alternativa, son diferentes las medias de las muestras")
    else:
        print(f"P-valor {p_valor}, por tanto, no se puede aceptar la hipótesis alternativa, no podemos decir que las medias de las muestras sean diferentes")


#============================
# T-test para dos muestras con mayor o menor
#============================
# Cuando quiero comparar las medias de dos muestras
# Ho media 1 <= media 2
# Ha media 1 > media 2

def t_test_dos_muestras(muestra1, muestra2, target_numerica, signo_alternativa, umbral = 0.05):
    '''
    Cuando quiero comprobar si dos muestras tienen misma media
    Ho no hay diferencia entre las medias de las muestras
    Ha sí hay diferencia entre las medias de las muestras
    target_numerica es el nombre de la columna de la muestra que quiero estudiar
    signo_alternativa puede ser "less" o "greater"
    '''
    p_valor = stats.ttest_ind(a=muestra1[target_numerica],b=muestra2[target_numerica],alternative=signo_alternativa).pvalue
    if p_valor < umbral:
        print(f"P-valor {p_valor}, por tanto, se acepta la hipótesis alternativa, media 1 es {signo_alternativa} que media 2")
    else:
        print(f"P-valor {p_valor}, por tanto, no se puede aceptar la hipótesis alternativa, no podemos decir que media 1 es {signo_alternativa} que media 2")



# 3. Medidas estadísticas variables categóricas

In [ ]:
#============================
# Z-test para porcentaje de supervivencia de una muestra de una variable categórica
#============================
def z_test_categorica(muestra, target_categorica, valor_analisis, tasa_supervivencia_poblacional_deseada):
    '''
    muestra conjunto de datos de la muestra
    target_categorica es la columna sobre la que quiero estudiar la proporcición de veces que aparece un valor
    valor_analisis es el valor de la target que quiero estudiar su proporción
    tasa_superviviencia_poblacional_deseada es el valor poblacional con el que quiero comparar la proporción muestral que se va a calcular
    '''

    from statsmodels.stats.proportion import proportions_ztest

    tasa_supervivencia_muestral = round(muestra[target_categorica].value_counts(normalize = True)[valor_analisis] * 100,2)/100
    n = len(muestra)       # Tamaño de la muestra
    # Calcular el estadístico Z y el p-valor
    z_stat, p_value = proportions_ztest(tasa_supervivencia_muestral * n, n, tasa_supervivencia_poblacional_deseada)

    # Mostrar resultados
    print(f"Estadístico Z: {z_stat}")
    print(f"P-valor: {p_value}")

    if p_value < 0.05:
        print(f"Como p_valor < 0.05, se rechaza la hipótesis nula de que la supervivencia poblacional es {tasa_supervivencia_poblacional_deseada}")
    else:
        print(f"Como p_valor >= 0.05, no se puede rechazar la hipótesis nula de que la supervivencia poblacional es {tasa_supervivencia_poblacional_deseada}")


#============================
# Z-test para umbrales de porcentajes de supervivencia de una muestra de una variable categórica
#============================
def z_test_umbrales_categorica(muestra, target_categorica, valor_analisis, tasa_supervivencia_poblacional_deseada, signo_alternativa):
    '''
    muestra conjunto de datos de la muestra
    target_categorica es la columna sobre la que quiero estudiar la proporcición de veces que aparece un valor
    valor_analisis es el valor de la target que quiero estudiar su proporción
    tasa_superviviencia_poblacional_deseada es el valor poblacional con el que quiero comparar la proporción muestral que se va a calcular
    signo_alternativa tiene que ser "smaller" o "larger"
    '''

    from statsmodels.stats.proportion import proportions_ztest

    tasa_supervivencia_muestral = round(muestra[target_categorica].value_counts(normalize = True)[valor_analisis] * 100,2)/100
    n = len(muestra)       # Tamaño de la muestra
    # Calcular el estadístico Z y el p-valor
    z_stat, p_value = proportions_ztest(tasa_supervivencia_muestral * n, n, tasa_supervivencia_poblacional_deseada, alternative = signo_alternativa)

    # Mostrar resultados
    print(f"Estadístico Z: {z_stat}")
    print(f"P-valor: {p_value}")

    if p_value < 0.05:
        print(f"Como p_valor < 0.05, se rechaza la hipótesis nula de que la supervivencia poblacional es {signo_alternativa} que {tasa_supervivencia_poblacional_deseada}")
    else:
        print(f"Como p_valor >= 0.05, no se puede rechazar la hipótesis nula de que la supervivencia poblacional es {signo_alternativa} que {tasa_supervivencia_poblacional_deseada}")


#============================
# Z-test para comparar supervivencia de una cetgórica en dos muestras
#============================


# 4. Probabilidad

In [ ]:
#============================
# Probabilidad
#============================
def prob(data, target_categorica, valor_analisis):
    casos_favorables = len(data.loc[data[target_categorica] == valor_analisis])
    casos_totales = len(data)
    print(f"Probabilidad de que {target_categorica} sea {valor_analisis}", casos_favorables/casos_totales)

#============================
# Probabilidad intersección
#============================
def prob_interseccion(data, target_categorica1, valor1, target_categorica2, valor2):
    cond1 = data[target_categorica1] == valor1
    cond2 = data[target_categorica2] == valor2
    casos_fav = len(data[cond1|cond2])
    casos_tot = len(data)
    print(f"Probabilidad de que {target_categorica1} sea {valor1} y {target_categorica2} sea {valor2} es ", casos_fav/casos_tot)

#============================
# Probabilidad unión
#============================
def prob_union(data, target_categorica1, valor1, target_categorica2, valor2):
    cond1 = data[target_categorica1] == valor1
    cond2 = data[target_categorica2] == valor2
    casos_fav = len(data[cond1&cond2])
    casos_tot = len(data)
    print(f"Probabilidad de que {target_categorica1} sea {valor1} o {target_categorica2} sea {valor2} es ", casos_fav/casos_tot)

#============================
# Probabilidad condicionada
#============================
def prob_cond(data, target_categorica1, valor1, target_categorica2_condicionante, valor2):
    cond1 = data[target_categorica1] == valor1
    cond2 = data[target_categorica2_condicionante] == valor2
    casos_fav = len(data[cond1&cond2])
    casos_tot = len(data[cond2])
    print(f"Probabilidad de que {target_categorica1} sea {valor1} sabiendo que {target_categorica2_condicionante} es {valor2} es ", casos_fav/casos_tot)

